In [4]:
pip install pdfminer


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import *
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage,PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
import os


In [8]:
def parsePDF(pdf_path,txt_path):
    # Open a pdf document in binary read mode
    fp = open(pdf_path,'rb')

    # Use the file object to create a pdf document analyzer
    parser = PDFParser(fp)

    # The object of the pdf document is connected to the analyzer
    doc = PDFDocument(parser=parser)
    parser.set_document(doc=doc)

    # If it is an encrypted pdf, enter the password. The new version does not appear to have this attribute
    # doc._initialize_password()

    # Create pdf Explorer to manage shared resources
    resource = PDFResourceManager()

    # Parameter analyzer
    laparam=LAParams()

    # Create an aggregator
    device = PDFPageAggregator(resource,laparams=laparam)

    # Create a pdf page interpreter
    interpreter = PDFPageInterpreter(resource,device)
    
    # Get a collection of pages
    for page in PDFPage.get_pages(fp):
        # Use the page interpreter to read
        interpreter.process_page(page)
        # Use aggregators to get content
        layout = device.get_result()
        
        for x in layout:
            if isinstance(x, LTTextBoxHorizontal):  # Get text content
                    # Save text content
                with open(txt_path, 'a',encoding='UTF-8',errors='ignore') as f:
                    results = x.get_text()

                    f.write(results + '\n')


In [19]:
import os

script_directory = "/Users/nicho/Documents/Hong Kong/HKUST/UROP/TNFD/Report"

# Loop through the years
for year in range(2022, 2026):
    i = 1
    year_folder = os.path.join(script_directory, str(year))
    pdf_folder = os.path.join(year_folder, 'pdf')
    txt_folder = os.path.join(year_folder, 'txt')

    # Check if the 'pdf' folder exists for the current year
    if os.path.exists(pdf_folder) and os.path.isdir(pdf_folder):
        # Create 'txt' folder if it doesn't exist
        if not os.path.exists(txt_folder):
            os.makedirs(txt_folder)

        # Rename PDF files: replacing '+' with ' '
        for filename in os.listdir(pdf_folder):
            new_filename = filename.replace('+', ' ')
            old_file_path = os.path.join(pdf_folder, filename)
            new_file_path = os.path.join(pdf_folder, new_filename)
            if old_file_path != new_file_path:  # Check if renaming is needed
                os.rename(old_file_path, new_file_path)

        # Loop through renamed PDF files in the 'pdf' folder
        for pdf_filename in os.listdir(pdf_folder):
            pdf_path = os.path.join(pdf_folder, pdf_filename)

            # Now you can perform operations on each PDF file
            txt_path = os.path.join(txt_folder, f"{pdf_filename.rstrip('.pdf')}.txt")
            try:
                parsePDF(pdf_path, txt_path)
            except Exception as e:
                print(f'Error processing {pdf_filename} in {year}: {e}')
                continue
            print(f"Finished processing {i} files")
            i += 1
    else:
        print(f"Folder not found for {year}: {pdf_folder}")

Folder not found for 2022: /Users/nicho/Documents/Hong Kong/HKUST/UROP/TNFD/Report/2022/pdf
Finished processing 1 files
Finished processing 2 files
Finished processing 3 files
Error processing .DS_Store in 2023: No /Root object! - Is this really a PDF?
Finished processing 4 files
Finished processing 5 files
Finished processing 6 files
Finished processing 7 files
Finished processing 8 files
Finished processing 9 files
Finished processing 10 files
Finished processing 11 files
Finished processing 1 files
Finished processing 2 files
Finished processing 3 files
Finished processing 4 files
Finished processing 5 files
Finished processing 6 files
Error processing Fujifilm Holdings_2024_Nature TNFD Report.pdf in 2024: Unsupported revision: param={'CF': {'StdCF': {'AuthEvent': /'DocOpen', 'CFM': /'AESV3', 'Length': 32}}, 'Filter': /'Standard', 'Length': 256, 'O': b'\\\xfbKeW)2\xb3\x9e{[-*|\x8bF\x87\x14u\x99\x8d\x1c6\x9a\xa0;\xec\x88\xf1"\xc1\xd4\x97\x7f"\x8d\xd0\xa8;\xe7\x8a7rz\xe0Z\xb8w\x00\x00

In [10]:
import os
import pandas as pd

script_directory = "/Users/nicho/Documents/Hong Kong/HKUST/UROP/TNFD/Report"

# Define the years
years = list(range(2023, 2026))

# Create an empty list to store DataFrames
dfs = []

# Loop through the years
for year in years:
    # Define the paths for 'pdf' and 'txt' folders
    pdf_folder = os.path.join(script_directory, str(year), 'pdf')
    txt_folder = os.path.join(script_directory, str(year), 'txt')

    # Get the set of filenames in 'txt' folder without the '.txt' extension
    txt_filenames_set = set([filename[:-4] for filename in os.listdir(txt_folder) if filename.endswith('.txt')])

    # Create a DataFrame for the current year
    df = pd.DataFrame({
        'Year': [year] * len(os.listdir(pdf_folder)),
        'Filename': [pdf_filename[:-4] for pdf_filename in os.listdir(pdf_folder)],
        'HasTxtFiles': [1 if pdf_filename[:-4] in txt_filenames_set else 0 for pdf_filename in os.listdir(pdf_folder)]
    })

    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
result_df = pd.concat(dfs, ignore_index=True)

# Display the resulting DataFrame
print(result_df)
result_df.to_excel('/Users/nicho/Documents/Hong Kong/HKUST/UROP/TNFD/1.PDFandTXT.xlsx',index=False)

    Year                                           Filename  HasTxtFiles
0   2023  Robeco_2023_Mainstream Annual Report Financial...            1
1   2023         Nissui Corporation_2023_Nature TNFD Report            1
2   2023     Cellnex Telecom_2023_Sustainability ESG Report            1
3   2023                                              .DS_S            0
4   2023     CTBC Financial Holding_2023_Nature TNFD Report            1
..   ...                                                ...          ...
67  2025     Rhätische Bahn_2025_Sustainability ESG Report            1
68  2025   Glenmark Pharmaceuticals_2025_Nature TNFD Report            1
69  2025  United Overseas Bank_2025_Sustainability ESG R...            1
70  2025                 AMEA Power_2025_Nature TNFD Report            1
71  2025           Environment Bank_2025_Nature TNFD Report            1

[72 rows x 3 columns]
